## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-27-23-50-48 +0000,nypost,Athletic event that allows steroids sues World...,https://nypost.com/2025/08/27/sports/athletic-...
1,2025-08-27-23-49-39 +0000,nyt,A Traumatized Minneapolis Confronts Another Tr...,https://www.nytimes.com/2025/08/27/us/minneapo...
2,2025-08-27-23-47-40 +0000,startribune,Timeline of mass shooting at Minneapolis church,https://www.startribune.com/timeline-annunciat...
3,2025-08-27-23-46-41 +0000,nypost,Manhattan-sized interstellar object is covered...,https://nypost.com/2025/08/27/world-news/manha...
4,2025-08-27-23-45-42 +0000,bbc,Blair joins White House meeting with Trump on ...,https://www.bbc.com/news/articles/c4gmrxd8ryno...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
39,trump,53
42,gaza,14
20,shooting,13
14,minneapolis,13
304,new,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
343,2025-08-27-00-11-00 +0000,wsj,President Trump’s attempt to fire Fed governor...,https://www.wsj.com/politics/policy/trumps-fed...,123
159,2025-08-27-16-15-57 +0000,nypost,Cracker Barrel changed logo after call with Wh...,https://nypost.com/2025/08/27/business/cracker...,99
201,2025-08-27-13-51-00 +0000,nypost,Trump administration announces it’s going to r...,https://nypost.com/2025/08/27/us-news/trump-ad...,96
195,2025-08-27-14-30-00 +0000,wsj,A Beijing-linked espionage campaign that hit U...,https://www.wsj.com/politics/national-security...,94
130,2025-08-27-18-21-00 +0000,wsj,President Trump’s tariff rollout has taken num...,https://www.wsj.com/economy/trade/trump-tariff...,92


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
343,123,2025-08-27-00-11-00 +0000,wsj,President Trump’s attempt to fire Fed governor...,https://www.wsj.com/politics/policy/trumps-fed...
98,72,2025-08-27-20-33-06 +0000,nypost,Minneapolis mayor uses Catholic school shootin...,https://nypost.com/2025/08/27/us-news/minneapo...
66,52,2025-08-27-21-29-33 +0000,nypost,Zohran Mamdani would easily win NYC mayor race...,https://nypost.com/2025/08/27/us-news/zohran-m...
81,47,2025-08-27-21-01-42 +0000,nypost,Angry Cracker Barrel workers dish on skimpy pa...,https://nypost.com/2025/08/27/business/angry-c...
298,44,2025-08-27-07-04-00 +0000,wsj,Israeli Troops Targeted a Camera in Gaza Hospi...,https://www.wsj.com/world/middle-east/israeli-...
201,43,2025-08-27-13-51-00 +0000,nypost,Trump administration announces it’s going to r...,https://nypost.com/2025/08/27/us-news/trump-ad...
237,42,2025-08-27-10-59-06 +0000,cbc,Some FEMA staff who signed dissent letter over...,https://www.cbc.ca/news/world/fema-letter-diss...
231,42,2025-08-27-11-15-50 +0000,cbc,Denmark summons top U.S. diplomat after report...,https://www.cbc.ca/news/world/denmark-us-green...
50,42,2025-08-27-22-00-06 +0000,nypost,NYC jewelers insist Travis Kelce could’ve gott...,https://nypost.com/2025/08/27/us-news/nyc-jewe...
332,40,2025-08-27-01-00-00 +0000,wsj,The first Black woman to serve as a Federal Re...,https://www.wsj.com/economy/central-banking/li...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
